<a href="https://colab.research.google.com/github/zyythn/mini-project-ic/blob/main/Hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import random

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/heart_attack_data.csv')

In [ ]:
X = data.drop('Output', axis=1).values
y = data['Output'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Define the ANN model architecture
def create_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [ ]:
def fitness_function(individual):
    model = create_model()
    model.set_weights(individual)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    fitness = model.evaluate(X_train, y_train, verbose=0)[1]
    return fitness

In [ ]:
def initialize_population(population_size):
    population = []
    for _ in range(population_size):
        individual = create_model().get_weights()
        population.append(individual)
    return population

def crossover(parent1, parent2):
    child = []
    for i in range(len(parent1)):
        if random.random() < 0.5:
            child.append(parent1[i])
        else:
            child.append(parent2[i])
    return child

def mutate(individual, mutation_rate):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            individual[i] += np.random.normal(0, 0.1, individual[i].shape)
    return individual

def select_parents(population, fitness_values):
    fitness_sum = sum(fitness_values)
    probabilities = [fitness / fitness_sum for fitness in fitness_values]
    parent1 = random.choices(population, probabilities)[0]
    parent2 = random.choices(population, probabilities)[0]
    return parent1, parent2

def evolve_population(population, fitness_values, elite_size, mutation_rate):
    new_population = []
    sorted_indices = np.argsort(fitness_values)[::-1]
    elite = [population[i] for i in sorted_indices[:elite_size]]
    new_population.extend(elite)

    while len(new_population) < len(population):
        parent1, parent2 = select_parents(population, fitness_values)
        child = crossover(parent1, parent2)
        child = mutate(child, mutation_rate)
        new_population.append(child)

    return new_population

In [ ]:
population_size = 50
elite_size = 5
mutation_rate = 0.1
generations = 10

population = initialize_population(population_size)

for generation in range(generations):
    fitness_values = [fitness_function(individual) for individual in population]
    population = evolve_population(population, fitness_values, elite_size, mutation_rate)
    best_individual = population[0]
    best_fitness = fitness_function(best_individual)
    print(f"Generation {generation+1}: Best Fitness = {best_fitness}")

best_model = create_model()
best_model.set_weights(best_individual)

Generation 1: Best Fitness = 0.6348074078559875
Generation 2: Best Fitness = 0.6407988667488098
Generation 3: Best Fitness = 0.6407988667488098
Generation 4: Best Fitness = 0.6409415006637573
Generation 5: Best Fitness = 0.6416547894477844
Generation 6: Best Fitness = 0.6413694620132446
Generation 7: Best Fitness = 0.6417974233627319
Generation 8: Best Fitness = 0.6407988667488098
Generation 9: Best Fitness = 0.6420827507972717
Generation 10: Best Fitness = 0.6409415006637573


In [ ]:
print(f"Best Fitness: {best_fitness*100:.2f}%")

Best Fitness: 64.09%
